## 1. General functions

In [1]:
import os.path as osp
import os
import errno
import cv2
from matplotlib import pyplot as plt
import numpy as np
import json
from collections import namedtuple

In [2]:
def createValidPathnames(root_path, part_path, images_path):
    """
        returns a tuple containing the three needed pathnames
        contains a check if the file is there
    """
    labelsGrayscale_path = osp.join(root_path, part_path, "labelsGrayscale")
    if os.path.isdir(labelsGrayscale_path):
        print("Found grayscale label folder.")
    else:
        raise FileNotFoundError(errno.ENOENT, os.strerror(errno.ENOENT), labelsGrayscale_path)

    # labels RGB path
    labelsRGB_path = osp.join(root_path, part_path, "labelsRGB")
    if os.path.isdir(labelsRGB_path):
        print("Found RGB label folder.")
    else:
        raise FileNotFoundError(errno.ENOENT, os.strerror(errno.ENOENT), labelsRGB_path)

    # color mapping path
    colorMapping_path = osp.join(root_path, part_path, "ColorMapping.json" )
    if os.path.isdir(labelsGrayscale_path):
        print("Found color mapping file.")
    else:
        raise FileNotFoundError(errno.ENOENT, os.strerror(errno.ENOENT), colorMapping_path)
    return labelsGrayscale_path, labelsRGB_path, colorMapping_path

In [3]:
def createColorMapping(json_path):
    """
        reads a json file and creates a list of object containing the transformation details
        eg.
        colorMapping = [ColorMapping(Class='Spalling', RGB=[255, 0, 0], Grayscale=25),
         ColorMapping(Class='Rust Stain', RGB=[165, 42, 42], Grayscale=120), ... ]
         Access the element:
         colorMapping[0].RGB -> [255,0,0]
    """
    colorMapping = []
    with open(colorMapping_path) as json_file:
        json_file = json_file.read()
        json_file = json_file.split(";")
        for data in json_file:
            colorMapping.append(json.loads(data, object_hook=lambda d: namedtuple('ColorMapping', d.keys())(*d.values())))
    return colorMapping

## 2. Transform RGB image to specific Grayscale mappings

In [ ]:
# Create paths
root_path = "S2DSdevkit"
part_path = "part2"
images_path = "images"

labelsGrayscale_path, labelsRGB_path, colorMapping_path = createValidPathnames(root_path, part_path, images_path)

In [4]:
# Create Color Mapping Part 2
colorMapping = createColorMapping(colorMapping_path)
colorMapping

NameError: name 'colorMapping_path' is not defined

In [ ]:
# create list of files in the folder
list_paths = os.listdir(labelsRGB_path)
# get only images ending on .jpg
list_labels_rgb_with_postfix = list((filter(lambda k: '.png' in k, list_paths)))
# remove the file ending .jpg
list_labels_rgb_without_postfix = list((map(lambda k: osp.splitext(k)[0], list_labels_rgb_with_postfix)))
# show first five elements
list_labels_rgb_without_postfix[:5]

In [ ]:
"""def RGBtoGray(RGBimg, colorMapping):
    RGBimg = cv2.cvtColor(RGBimg, cv2.COLOR_BGR2RGB)
    row = RGBimg.shape[0]
    col = RGBimg.shape[1]
    GRAYimg = np.zeros((row, col))
    for x in range(0,row):
        for y in range(0,col):
            pixel = RGBimg[x,y,:]
            for cm in colorMapping:
                if np.array_equal(pixel, np.array(cm.RGB)):
                    GRAYimg[x,y] = cm.Grayscale    
    return GRAYimg    """

In [ ]:
def map_RGB2grayscale(img, colors, vals, invalid_val=0):
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    s = 256**np.arange(3)
    img1D = img.reshape(-1,img.shape[2]).dot(s)
    colors1D = colors.reshape(-1,img.shape[2]).dot(s)
    sidx = colors1D.argsort()
    idx0 = np.searchsorted(colors1D, img1D, sorter=sidx)
    idx0[idx0==len(sidx)] = 0
    mapped_idx = sidx[idx0]
    valid = colors1D[mapped_idx] == img1D
    return np.where(valid, vals[mapped_idx], invalid_val).reshape(img.shape[:2])

In [ ]:
colors = []
vals = []
for element in colorMapping:
    colors.append(element.RGB)
    vals.append(element.Grayscale)
    
colors = np.array(colors)
vals = np.array(vals)

In [ ]:
for name_img in list_labels_rgb_without_postfix:
    print(".", end = '')
    origin_label_path = osp.join(labelsRGB_path, name_img + ".png")
    target_label_path = osp.join(labelsGrayscale_path, name_img + ".png")
    image = cv2.imread(origin_label_path)
    #gray = RGBtoGray(image, colorMapping)
    gray = map_RGB2grayscale(image, colors, vals)
    cv2.imwrite(target_label_path, gray)

## 3. Transform Grayscale image to specific RGB mappings

In [5]:
# Create paths
root_path = "S2DSdevkit"
part_path = "part1"
images_path = "images"

labelsGrayscale_path, labelsRGB_path, colorMapping_path = createValidPathnames(root_path, part_path, images_path)

Found grayscale label folder.
Found RGB label folder.
Found color mapping file.


In [6]:
# create list of files in the folder
list_paths = os.listdir(labelsGrayscale_path)
# get only images ending on .jpg
list_labels_grayscale_with_postfix = list((filter(lambda k: '.png' in k, list_paths)))
# remove the file ending .jpg
list_labels_grayscale_without_postfix = list((map(lambda k: osp.splitext(k)[0], list_labels_grayscale_with_postfix)))
# show first five elements
list_labels_grayscale_without_postfix[:5]

['4530_541__0_prufung_2008H_SCHADEN_RISS MIT AUSSINTERUNG AM WL 2',
 '5038_547__1_prufung_2005H1_SCHADEN_FEUCHTER RISS MIT  AUSBLÜHUNG',
 '4429_776__0_PRUFUNG_2005H_SCHADEN_BILD_5 - QUERRISS IN AUFLAGERBANK',
 '5038_550__2_PRUFUNG_2014E_SCHADEN_2011_H_LÄNGSRISS IN HINTERER KAMMERWAND',
 '5038_820__0_PRUFUNG_2006H_SCHADEN_BILD 4, QUERRISSE IM GESIMS']

In [8]:
# using colormapping from part 2
# colorMapping, colors, vals
# Create Color Mapping Part 2
colorMapping = createColorMapping(colorMapping_path)
colorMapping

[ColorMapping(Class='Spalling', RGB=[255, 0, 0], Grayscale=25),
 ColorMapping(Class='Rust Stain', RGB=[165, 42, 42], Grayscale=120),
 ColorMapping(Class='Discoloration', RGB=[0, 255, 255], Grayscale=127),
 ColorMapping(Class='Vegatation', RGB=[127, 255, 0], Grayscale=50),
 ColorMapping(Class='Crack', RGB=[255, 255, 255], Grayscale=159),
 ColorMapping(Class='Graffiti', RGB=[128, 0, 128], Grayscale=90)]

In [9]:
def map_grayscale2RGB(img, colors, vals, invalid_val=0, CHANNELS = 3):
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img1D = img.reshape(-1)
    sidx = vals.argsort()
    idx0 = np.searchsorted(vals, img1D, sorter=sidx)
    idx0[idx0==len(sidx)] = 0
    mapped_idx = sidx[idx0]
    valid = vals[mapped_idx] == img1D
    red = np.where(valid, colors[:,0][mapped_idx], invalid_val).reshape(img.shape[0],img.shape[1])
    green = np.where(valid, colors[:,1][mapped_idx], invalid_val).reshape(img.shape[0],img.shape[1])
    blue = np.where(valid, colors[:,2][mapped_idx], invalid_val).reshape(img.shape[0],img.shape[1])
    # Opencv saves in bgr
    return np.dstack((blue,green,red))

In [10]:
"""# test with one img
origin_label_1_path = osp.join(labelsGrayscale_path, list_labels_grayscale_without_postfix[0] + ".png")
target_label_1_path = osp.join(labelsRGB_path, list_labels_grayscale_without_postfix[0] + ".png")
img = cv2.imread(origin_label_1_path)
#grey = np.array([[159,  90,  25,  50],[0,  50, 120,   0]])
rgb = map_grayscale2RGB(img, colors, vals)
cv2.imwrite(target_label_1_path, rgb)"""

'# test with one img\norigin_label_1_path = osp.join(labelsGrayscale_path, list_labels_grayscale_without_postfix[0] + ".png")\ntarget_label_1_path = osp.join(labelsRGB_path, list_labels_grayscale_without_postfix[0] + ".png")\nimg = cv2.imread(origin_label_1_path)\n#grey = np.array([[159,  90,  25,  50],[0,  50, 120,   0]])\nrgb = map_grayscale2RGB(img, colors, vals)\ncv2.imwrite(target_label_1_path, rgb)'

In [12]:
colors = []
vals = []
for element in colorMapping:
    colors.append(element.RGB)
    vals.append(element.Grayscale)
    
colors = np.array(colors)
vals = np.array(vals)

In [13]:
for name_img in list_labels_grayscale_without_postfix:
    print(".", end = '')
    origin_label_path = osp.join(labelsGrayscale_path, name_img + ".png")
    target_label_path = osp.join(labelsRGB_path, name_img + ".png")
    image = cv2.imread(origin_label_path)
    gray = map_grayscale2RGB(image, colors, vals)
    cv2.imwrite(target_label_path, gray)

.........................................................................................................................................................